In [ ]:
from nbdev import *default_exp individual

In [ ]:
#hideimport syssys.path.append("..")import jsonimport numpy as npimport tensorflow as tffrom tensorflow.keras.models import Modelfrom tensorflow.keras.layers import Input, Dense, Lambda, Multiplyimport gym

# DHPCTIndividual> Class for implementing Deep Perceptual Control Theory individuals.

In [ ]:
#exportclass DHPCTIndividual:    """DHPCTIndividual encapsulates an environment and a keras model representing a PCT hierarchy."""        def __init__(self, env_name, env_props=None, levels=None, activation_funcs=None, weight_types=None):        """        Initialize a new individual with environment and hierarchy specifications.                Parameters:        - env_name: String identifier for the environment        - env_name: Gym environment ID (e.g. 'CartPole-v1')        - env_props: Optional dictionary of environment properties        - levels: List of integers representing nodes per level, from perception to action        - activation_funcs: List of activation functions for each level        - weight_types: List of weight initialization types for each level        """        self.env_name = env_name        self.env_name = env_name        self.env_props = env_props or {}                # Default configuration if not provided        self.levels = levels or [8, 4, 2]        self.activation_funcs = activation_funcs or ['tanh', 'tanh', 'tanh']        self.weight_types = weight_types or ['normal', 'normal', 'normal']                # Computed properties        self.num_levels = len(self.levels)        self.last_reward = 0        self.fitness = float('-inf')        self.age = 0        self.births = 0        self.params = {}                # Create the OpenAI Gym environment        self.env = gym.make(self.env_name)                # Extract observation and action space dimensions        self.obs_dim = self.env.observation_space.shape[0]        if hasattr(self.env.action_space, 'n'):            self.action_dim = self.env.action_space.n            self.discrete_actions = True        else:            self.action_dim = self.env.action_space.shape[0]            self.discrete_actions = False                # Initialize the model        self.model = self._build_model()        def _build_model(self):        """        Build a Keras model representing a PCT hierarchy.                Returns:        - A compiled Keras model        """        # Input layer for observations        inputs = Input(shape=(self.obs_dim,), name='observations')                # Build the hierarchy        x = inputs        for i in range(self.num_levels):            # Perception layer at this level            x = Dense(self.levels[i],                      activation=self.activation_funcs[i],                     kernel_initializer=self.weight_types[i],                     name=f'perception_L{i}')(x)                        # Reference signals (if not the top level)            if i < self.num_levels - 1:                # Error calculation (perception - reference)                x = Lambda(lambda x: x, name=f'error_L{i}')(x)                # Output layer for actions        if self.discrete_actions:            outputs = Dense(self.action_dim, activation='softmax', name='actions')(x)        else:            outputs = Dense(self.action_dim, activation='tanh', name='actions')(x)                # Create and compile model        model = Model(inputs=inputs, outputs=outputs)        model.compile(optimizer='adam', loss='mse')                return model        def get_action(self, observation):        """        Get an action from the model given an observation.                Parameters:        - observation: The current environment observation                Returns:        - The selected action        """        # Reshape observation for model input        obs = np.array(observation).reshape(1, -1)                # Get model prediction        action_probs = self.model.predict(obs, verbose=0)[0]                if self.discrete_actions:            # For discrete actions, sample from probability distribution            action = np.random.choice(self.action_dim, p=action_probs)        else:            # For continuous actions, use the output directly            action = action_probs                return action        def run_episode(self, max_steps=1000, render=False):        """        Run a complete episode in the environment.                Parameters:        - max_steps: Maximum number of steps per episode        - render: Whether to render the environment                Returns:        - Total episode reward        """        observation, info = self.env.reset()        total_reward = 0        done = False        truncated = False        step = 0                while not (done or truncated) and step < max_steps:            if render:                self.env.render()                        # Get action from model            action = self.get_action(observation)                        # Take action in environment            observation, reward, done, truncated, info = self.env.step(action)                        total_reward += reward            step += 1                self.last_reward = total_reward        self.fitness = max(self.fitness, total_reward)  # Update fitness if better        self.age += 1  # Increment age                return total_reward        def mate(self, other, mutation_rate=0.1):        """        Create offspring by combining weights with another individual.                Parameters:        - other: Another DHPCTIndividual to mate with        - mutation_rate: Probability of mutating each weight                Returns:        - A new DHPCTIndividual offspring        """        # Create new individual with same configuration        offspring = DHPCTIndividual(            self.env_name,             self.gym_name,            self.env_props,            self.levels,            self.activation_funcs,            self.weight_types        )                # Crossover: combine weights from parents        for i, layer in enumerate(offspring.model.layers):            if len(layer.get_weights()) > 0:  # Skip layers without weights                # Get parent weights                parent1_weights = self.model.layers[i].get_weights()                parent2_weights = other.model.layers[i].get_weights()                                offspring_weights = []                for w1, w2 in zip(parent1_weights, parent2_weights):                    # Crossover: randomly select weights from each parent                    mask = np.random.random(w1.shape) < 0.5                    new_weights = np.where(mask, w1, w2)                                        # Mutation: randomly perturb weights                    mutation_mask = np.random.random(w1.shape) < mutation_rate                    mutations = np.random.normal(0, 0.1, w1.shape)                    new_weights = np.where(mutation_mask, new_weights + mutations, new_weights)                                        offspring_weights.append(new_weights)                                # Set the combined weights to offspring                layer.set_weights(offspring_weights)                # Update birth records        self.births += 1        other.births += 1                return offspring        def mutate(self, mutation_rate=0.1, mutation_scale=0.1):        """        Mutate the weights of the individual.                Parameters:        - mutation_rate: Probability of mutating each weight        - mutation_scale: Scale of the mutations                Returns:        - Self, for method chaining        """        for layer in self.model.layers:            if len(layer.get_weights()) > 0:  # Skip layers without weights                weights = layer.get_weights()                new_weights = []                                for w in weights:                    # Create mutation mask and perturbations                    mutation_mask = np.random.random(w.shape) < mutation_rate                    mutations = np.random.normal(0, mutation_scale, w.shape)                                        # Apply mutations                    new_w = np.where(mutation_mask, w + mutations, w)                    new_weights.append(new_w)                                # Update weights                layer.set_weights(new_weights)                return self        def learn(self, observations, rewards, learning_rate=0.01, epochs=10):        """        Update model weights based on observations and rewards (online learning).                Parameters:        - observations: List of environment observations        - rewards: Corresponding rewards received        - learning_rate: Learning rate for weight updates        - epochs: Number of training epochs                Returns:        - Training history        """        # Convert to numpy arrays        observations = np.array(observations)        rewards = np.array(rewards)                # Get current action predictions        current_actions = self.model.predict(observations, verbose=0)                # Scale rewards to use as targets        reward_scale = np.abs(rewards).max() if np.abs(rewards).max() > 0 else 1        scaled_rewards = rewards / reward_scale                # Update action values based on rewards        target_actions = current_actions.copy()        for i in range(len(observations)):            if self.discrete_actions:                # For discrete actions, increase probability of rewarded actions                action_idx = np.argmax(current_actions[i])                target_actions[i, action_idx] += learning_rate * scaled_rewards[i]                # Normalize to ensure valid probability distribution                target_actions[i] = target_actions[i] / target_actions[i].sum()            else:                # For continuous actions, adjust action outputs based on reward                target_actions[i] += learning_rate * scaled_rewards[i]                # Clip to valid range                target_actions[i] = np.clip(target_actions[i], -1, 1)                # Train the model        history = self.model.fit(            observations, target_actions,            epochs=epochs,            verbose=0        )                return history        def save(self, filepath):        """        Save the individual's model and configuration.                Parameters:        - filepath: Path to save the model and configuration        """        # Save model weights        self.model.save_weights(filepath + '_weights.h5')                # Save configuration        config = {            'env_name': self.env_name,            'env_name': self.env_name,            'env_props': self.env_props,            'levels': self.levels,            'activation_funcs': self.activation_funcs,            'weight_types': self.weight_types,            'fitness': self.fitness,            'age': self.age,            'births': self.births,            'params': self.params        }                with open(filepath + '_config.json', 'w') as f:            json.dump(config, f)        @classmethod    def load(cls, filepath):        """        Load an individual from saved model and configuration.                Parameters:        - filepath: Path to the saved model and configuration                Returns:        - A DHPCTIndividual instance        """        # Load configuration        with open(filepath + '_config.json', 'r') as f:            config = json.load(f)                # Create individual with loaded configuration        individual = cls(            config['env_name'],            config['gym_name'],            config['env_props'],            config['levels'],            config['activation_funcs'],            config['weight_types']        )                # Load model weights        individual.model.load_weights(filepath + '_weights.h5')                # Restore other attributes        individual.fitness = config['fitness']        individual.age = config['age']        individual.births = config['births']        individual.params = config['params']                return individual